In [22]:
# --------------------------------------------
# IMPORT LIBRAIRIES
# --------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_rows', None)

# 1 - Génération des données

In [ ]:
# --------------------------------------------
# PARAMETRES DE SIMULATION
# --------------------------------------------
Nobs = 10000 # nombre d'observations
NX = 6 # nombre de covariables
PX = 1/2 # probabilité qu'une covariable vaille 1
alpha_tau = 1 #effet du traitement

alpha_eZ = [2,2,0,0,0,0] #effet des covariables sur le propensity score
alpha0_eZ = -np.sum(alpha_eZ)*PX #intercept du propensity score
alpha_eY = [-2,-2,0,0,0,0] #effet des covariables sur la variable d'intérêt
alpha0_eY = -np.sum(alpha_eY)*PX #intercept de la variable d'intérêt

# --------------------------------------------
# GENERATION DES COVARIABLES X
# --------------------------------------------
X = np.random.binomial(1,PX,(Nobs,NX))

# --------------------------------------------
# GENERATION DE Z
# --------------------------------------------
def sigmoid(x): # Définition de la fonction sigmoïde
    return 1 / (1 + np.exp(-x))

eZ = sigmoid(X.dot(alpha_eZ) + alpha0_eZ) # Génération du propensity score
Z = np.random.binomial(1,eZ)

# --------------------------------------------
# GENERATION DE Y
# --------------------------------------------
eY = sigmoid(X.dot(alpha_eY) + alpha_tau*Z + alpha0_eY)
Y = np.random.binomial(1,eY)

# --------------------------------------------
# Calcul de E(Y(1)) et E(Y(0))
# --------------------------------------------
Z1 = np.ones(Nobs)
Z0 = np.zeros(Nobs)

EY1 = sigmoid(X.dot(alpha_eY) + alpha_tau*Z1 + alpha0_eY)
EY0 = sigmoid(X.dot(alpha_eY) + alpha_tau*Z0 + alpha0_eY)

# --------------------------------------------
# MISE EN FORME ET EXPORT DES DONNEES
# --------------------------------------------
df = pd.DataFrame(X, columns=[f"X{i+1}" for i in range(NX)])  # noms des colonnes X1, X2, ...
df['Y'] = Y
df['Z'] = Z
df['EY1'] = EY1
df['EY0'] = EY0

# Réorganiser les colonnes pour avoir Y, Z, puis les X
df = df[['Y', 'EY1', 'EY0', 'Z'] + [f"X{i+1}" for i in range(NX)]]

# 2 - Analyse des données

In [24]:
# Calculer la moyenne de Y pour chaque valeur de Z
mean_Y_by_Z = df.groupby('Z')['Y'].mean()

# Afficher les résultats
print('E[Y,Z=1] - E[Y,Z=0] =', round(mean_Y_by_Z[1]-mean_Y_by_Z[0],ndigits=3))   
tau = df['EY1'].mean()-df['EY0'].mean()
print('tau = E[Y(1)] - E[Y(0)] =', round(tau ,ndigits=3))

E[Y,Z=1] - E[Y,Z=0] = -0.085
tau = E[Y(1)] - E[Y(0)] = 0.171


# 3 - Modélisation

In [25]:
# --------------------------------------------
# Modèle 1 : moyenne par strate de X
# --------------------------------------------

# Regrouper par (X1, X2, Z) et calculer la moyenne de Y et l'effectif
grouped = df.groupby(['X1', 'X2', 'Z']).agg(
    mean_Y=('Y', 'mean'),
    count=('Y', 'count')
).reset_index()

# Réorganiser les résultats pour avoir les colonnes séparées pour Z=0 et Z=1
pivoted = grouped.pivot(index=['X1', 'X2'], columns='Z', values=['mean_Y', 'count'])

# Renommer les colonnes pour plus de clarté
pivoted.columns = ['E[Y|Z=0]', 'E[Y|Z=1]', 'N(Z=0)', 'N(Z=1)']
pivoted = pivoted.fillna(0)  # Remplir les valeurs NaN si une combinaison (X1, X2, Z) est absente

# Afficher le résultat final
print(pivoted)
print('')

# Calcul de la différence E[Y | Z=1] - E[Y | Z=0]
pivoted['Diff'] = pivoted['E[Y|Z=1]'] - pivoted['E[Y|Z=0]']

# Calcul du poids total N(Z=0) + N(Z=1)
pivoted['Poids'] = pivoted['N(Z=0)'] + pivoted['N(Z=1)']

# Calcul de la moyenne pondérée
moyenne_ponderee = (pivoted['Diff'] * pivoted['Poids']).sum() / pivoted['Poids'].sum()

# Affichage des résultats
print(pivoted)
print("\nMoyenne pondérée de la différence :", round(moyenne_ponderee, 3))

       E[Y|Z=0]  E[Y|Z=1]  N(Z=0)  N(Z=1)
X1 X2                                    
0  0   0.873596  0.950166  2136.0   301.0
   1   0.477918  0.719298  1268.0  1254.0
1  0   0.505645  0.732245  1240.0  1225.0
   1   0.102484  0.293256   322.0  2254.0

       E[Y|Z=0]  E[Y|Z=1]  N(Z=0)  N(Z=1)      Diff   Poids
X1 X2                                                      
0  0   0.873596  0.950166  2136.0   301.0  0.076571  2437.0
   1   0.477918  0.719298  1268.0  1254.0  0.241380  2522.0
1  0   0.505645  0.732245  1240.0  1225.0  0.226600  2465.0
   1   0.102484  0.293256   322.0  2254.0  0.190772  2576.0

Moyenne pondérée de la différence : 0.185
